# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [102]:
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [103]:
adm_pred = pd.read_csv(r"C:\Users\HP\Documents\IronHack\Bootcamp\IronHackLessons\Week2\LAB advanced Pandas\your-code\data\Admission_Predict.csv")
adm_pred

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [104]:
adm_pred.columns = adm_pred.columns.str.strip()
adm_pred.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [105]:
adm_pred.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [106]:
display(adm_pred.isnull().sum())



"""
if adm_pred.isna().any().any():
    print("True for missing values!")
else:
    print("False! They're arent missing values!")
    
"""

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

'\nif adm_pred.isna().any().any():\n    print("True for missing values!")\nelse:\n    print("False! They\'re arent missing values!")\n    \n'

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [107]:
adm_pred.set_index("Serial No.", drop = False) # OR inplace = True

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
381,381,324,110,3,3.5,3.5,9.04,1,0.82
382,382,325,107,3,3.0,3.5,9.11,1,0.84
383,383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [108]:
display(adm_pred.shape)

display(adm_pred.nunique())
display(adm_pred["GRE Score"].nunique()) 
display(adm_pred["CGPA"].nunique())


print(data.count())
display(adm_pred.head())
print(data.count())

#maybe this is the answer...
#adm_pred.set_index(["GRE Score", "CGPA"], drop=False, inplace = True) 

(385, 9)

Serial No.           385
GRE Score             49
TOEFL Score           29
University Rating      5
SOP                    9
LOR                    9
CGPA                 168
Research               2
Chance of Admit       60
dtype: int64

49

168

Serial No.           385
GRE Score            385
TOEFL Score          385
University Rating    385
SOP                  385
LOR                  385
CGPA                 385
Research             385
Chance of Admit      385
dtype: int64


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Serial No.           385
GRE Score            385
TOEFL Score          385
University Rating    385
SOP                  385
LOR                  385
CGPA                 385
Research             385
Chance of Admit      385
dtype: int64


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [109]:
adm_pred.set_index(["GRE Score", "CGPA"], drop = True, inplace = True) # it says to remove them aswell...
display(adm_pred.head())

,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [110]:
adm_pred.reset_index(inplace = True) 
display(adm_pred.head())

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [117]:
filter_rows = adm_pred[(adm_pred["CGPA"] > 9) & (adm_pred["Research"] == 1)]
display(filter_rows)
display(len(filter_rows))
display(filter_rows.count())

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
4,330,9.34,5,115,5,4.5,3.0,1,0.90
10,328,9.10,11,112,4,4.0,4.5,1,0.78
19,328,9.50,20,116,5,5.0,5.0,1,0.94
20,334,9.70,21,119,5,5.0,4.5,1,0.95
...,...,...,...,...,...,...,...,...,...
379,329,9.23,380,111,4,4.5,4.0,1,0.89
380,324,9.04,381,110,3,3.5,3.5,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84
382,330,9.45,383,116,4,5.0,4.5,1,0.91


101

GRE Score            101
CGPA                 101
Serial No.           101
TOEFL Score          101
University Rating    101
SOP                  101
LOR                  101
Research             101
Chance of Admit      101
dtype: int64

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [128]:
cgpa_sop = adm_pred[(adm_pred["CGPA"] > 9) & (adm_pred["SOP"] < 3.5)]
display(cgpa_sop)
cgpa_sop["Chance of Admit"].mean()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
28,338,9.40,29,118,4,3.0,4.5,1,0.91
62,327,9.02,63,114,3,3.0,3.0,0,0.61
140,326,9.11,141,114,3,3.0,3.0,1,0.83
217,324,9.01,218,111,4,3.0,3.0,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84


0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [142]:
def standardize(col):
    column = ((col)-np.mean(col)/np.std(col))    
    return column

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [149]:
adm_pred["CGPA_std"] = standardize(adm_pred["CGPA"])
adm_pred["GRE_std"] = standardize(adm_pred["GRE Score"])
adm_pred["LOR_std"] = standardize(adm_pred["LOR"])


display(adm_pred.head())

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,LOR_std,GRE_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,-4.640318,0.66959,309.563244
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-6.290318,-0.33041,288.563244
2,322,8.67,3,110,3,3.5,2.5,1,0.80,-5.620318,-1.33041,294.563244
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-6.080318,-0.83041,286.563244
4,330,9.34,5,115,5,4.5,3.0,1,0.90,-4.950318,-0.83041,302.563244


We will generate the decision choice at random using the code below. Please run the cell.

In [150]:
# Libraries
from random import choices

In [151]:
std_columns = ["CGPA_std", "GRE_std", "LOR_std"]

decision_choice = choices(std_columns, k=adm_pred.shape[0])
decision_choice

['LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [152]:
adm_pred["decision_choice"] = adm_pred.lookup(adm_pred.index, decision_choice)
display(adm_pred)

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,LOR_std,GRE_std,decision_choice
0,337,9.65,1,118,4,4.5,4.5,1,0.92,-4.640318,0.66959,309.563244,0.669590
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-6.290318,-0.33041,288.563244,288.563244
2,322,8.67,3,110,3,3.5,2.5,1,0.80,-5.620318,-1.33041,294.563244,-5.620318
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-6.080318,-0.83041,286.563244,-6.080318
4,330,9.34,5,115,5,4.5,3.0,1,0.90,-4.950318,-0.83041,302.563244,-0.830410
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,-5.250318,-0.33041,296.563244,-0.330410
381,325,9.11,382,107,3,3.0,3.5,1,0.84,-5.180318,-0.33041,297.563244,-5.180318
382,330,9.45,383,116,4,5.0,4.5,1,0.91,-4.840318,0.66959,302.563244,0.669590
383,312,8.78,384,103,3,3.5,4.0,0,0.67,-5.510318,0.16959,284.563244,-5.510318


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [153]:
adm_pred["decision"] = np.where(adm_pred["decision_choice"] > 0.8, 1, 0)
display(adm_pred.head())



"""
# with function

def decision(row):
    if row["deciding_choice"] > 0.8:
        return 1
    else: 
        return 0
        
adm_pred["decision"] = adm_pred.apply(decision, axis=1)

"""

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,LOR_std,GRE_std,decision_choice,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,-4.640318,0.66959,309.563244,0.669590,0
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-6.290318,-0.33041,288.563244,288.563244,1
2,322,8.67,3,110,3,3.5,2.5,1,0.80,-5.620318,-1.33041,294.563244,-5.620318,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-6.080318,-0.83041,286.563244,-6.080318,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,-4.950318,-0.83041,302.563244,-0.830410,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [162]:
print("Aplicants accepted: ", adm_pred["decision"].sum())



# len(adm_pred[adm_pred["decision"] == 1


"""
accepted_applicants = adm_pred[adm_pred["decision"] == 1]
accepted_applicants["decision"].count()

"""

Aplicants accepted:  133


'\naccepted_applicants = adm_pred[adm_pred["decision"] == 1]\naccepted_applicants["decision"].count()\n\n'

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [164]:
import re

display(adm_pred.head())

adm_pred.columns = adm_pred.columns.str.replace(" ","_", regex = True).str.replace("\W","", regex = True).str.lower()
display(adm_pred.head())

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,LOR_std,GRE_std,decision_choice,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,-4.640318,0.66959,309.563244,0.669590,0
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-6.290318,-0.33041,288.563244,288.563244,1
2,322,8.67,3,110,3,3.5,2.5,1,0.80,-5.620318,-1.33041,294.563244,-5.620318,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-6.080318,-0.83041,286.563244,-6.080318,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,-4.950318,-0.83041,302.563244,-0.830410,0


,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,lor_std,gre_std,decision_choice,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,-4.640318,0.66959,309.563244,0.669590,0
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-6.290318,-0.33041,288.563244,288.563244,1
2,322,8.67,3,110,3,3.5,2.5,1,0.80,-5.620318,-1.33041,294.563244,-5.620318,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-6.080318,-0.83041,286.563244,-6.080318,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,-4.950318,-0.83041,302.563244,-0.830410,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [171]:
# i ask help for this one, could really solve it, but i wasnt that far tho

adm_pred["adjusted_gre"] = np.where(adm_pred["university_rating"]>=4, adm_pred["gre_score"]+10, adm_pred["gre_score"])
pd.cut(adm_pred["adjusted_gre"], bins=4)

display(adm_pred.head())





"""

for x in range(adm_pred["gre_score"].count()):
    if adm_pred["university_rating"][x] >= 4:
        adm_pred["adjusted_gre"] = adm_pred["gre_score"] + 10

labels = ["low", "average", "high", "very high"]
bins = pd.cut(adm_pred["adjusted_gre"], 4, labels=labels)
bins.value_counts()

display(adm_pred.head())

"""

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,lor_std,gre_std,decision_choice,decision,adjusted_gre
0,347,9.65,1,118,4,4.5,4.5,1,0.92,-4.640318,0.66959,309.563244,0.669590,0,357
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-6.290318,-0.33041,288.563244,288.563244,1,316
2,322,8.67,3,110,3,3.5,2.5,1,0.80,-5.620318,-1.33041,294.563244,-5.620318,0,322
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-6.080318,-0.83041,286.563244,-6.080318,0,314
4,340,9.34,5,115,5,4.5,3.0,1,0.90,-4.950318,-0.83041,302.563244,-0.830410,0,350


'\n\nfor x in range(adm_pred["gre_score"].count()):\n    if adm_pred["university_rating"][x] >= 4:\n        adm_pred["adjusted_gre"] = adm_pred["gre_score"] + 10\n\nlabels = ["low", "average", "high", "very high"]\nbins = pd.cut(adm_pred["adjusted_gre"], 4, labels=labels)\nbins.value_counts()\n\ndisplay(adm_pred.head())\n\n'